In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

("Connection broken: ConnectionResetError(104, 'Connection reset by peer')", ConnectionResetError(104, 'Connection reset by peer'))


Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [2]:
sequence_to_classify = "За кого вы голосуете в 2020 году?"
# we can specify candidate labels in Russian or any other language above:
candidate_labels = ["Europe", "public health", "politics"]
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'За кого вы голосуете в 2020 году?',
 'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9048489928245544, 0.05722144991159439, 0.037929534912109375]}

In [1]:
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, load_metric

In [2]:
os.environ ["CUDA_VISIBLE_DEVICES"] = '0, 1'

In [3]:
class Args():
    model_name_or_path = "joeddav/xlm-roberta-large-xnli"
    cache_dir = "/home/work/xiaoyu/ckpt/xlm-roberta-large-xnli"
    data_cache_dir = "/home/work/xiaoyu/datasets/xnli"
    ignore_pad_token_for_loss = True
    per_device_eval_batch_size = 2
    val_max_target_length = None
    max_target_length = 128
    num_beams = None
    
args = Args()

In [4]:

nli_model = AutoModelForSequenceClassification.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
sequence_to_classify = "Handverlesene Romantische Hotels auf Mallorca. Elegant. Charmant. Einzigartig."
premise = sequence_to_classify
label = "Ein Hochzeitsreise Hotel ist die ideale Wahl für verliebte flitternde Paare oder solche, die den zweiten gemeinsamen Frühling erleben. ."
hypothesis = f'This example is {label}.'

In [27]:
x = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation_strategy='only_first')
#x = tokenizer(premise, hypothesis, return_tensors='pt', truncation_strategy='only_first')
logits = nli_model(x.to('cpu'))[0]

In [24]:
print(logits)

tensor([[-3.6847,  3.7045, -0.4683]], grad_fn=<AddmmBackward>)


In [25]:
entail_contradiction_logits = logits[:,[0,2]]
print(entail_contradiction_logits)
probs = entail_contradiction_logits.softmax(dim=1)
print(probs)
prob_label_is_true = probs[:,1]
print(prob_label_is_true)

tensor([[-3.6847, -0.4683]], grad_fn=<IndexBackward>)
tensor([[0.0386, 0.9614]], grad_fn=<SoftmaxBackward>)
tensor([0.9614], grad_fn=<SelectBackward>)
